imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

system info

In [2]:
!nvidia-smi

Sat Feb 26 02:51:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:03:00.0 Off |                  N/A |
| 41%   70C    P2    87W / 180W |   2595MiB /  8192MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ... WDDM  | 00000000:04:00.0 Off |                  N/A |
| 30%   

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3400765749523767564
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6475376230
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3535329487689808079
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


import data

In [4]:
bar = tqdm(total=4)
bar.set_description("Raspakek data")

inputLines = (np.load('traindata/inputLines.npy') / 255.0)
bar.update(1)

service = np.load('traindata/service.npy')
bar.update(1)

binaryData84 = np.load('traindata/binaryData84.npy')
bar.update(1)

#blockL0 = np.load('traindata/blockL0.npy')
#bar.update(1)
#
#blockR0 = np.load('traindata/blockR0.npy')
#bar.update(1)
#
#blockL1 = np.load('traindata/blockL1.npy')
#bar.update(1)
#
#blockR1 = np.load('traindata/blockR1.npy')
#bar.update(1)
#
#blockL2 = np.load('traindata/blockL2.npy')
#bar.update(1)
#
#blockR2 = np.load('traindata/blockR2.npy')
#bar.update(1)

#target_data = {'service': service, 'blockL0': blockL0, 'blockR0':blockR0, 'blockL1':blockL1, 'blockR1':blockR1, 'blockL2':blockL2, 'blockR2':blockR2}
target_data = {'service': service, 'binaryData84':binaryData84}
bar.update(1)
bar.close()

Raspakek data: 100%|██████████| 4/4 [00:02<00:00,  1.35it/s]


In [5]:
#remove all data after 100000 pos
if True:
    remAftr = 100000
    inputLines = inputLines[:remAftr]
    service = service[:remAftr]
    binaryData84 = binaryData84[:remAftr]
    #blockL0 = blockL0[:remAftr]
    #blockR0 = blockR0[:remAftr]
    #blockL1 = blockL1[:remAftr]
    #blockR1 = blockR1[:remAftr]
    #blockL2 = blockL2[:remAftr]
    #blockR2 = blockR2[:remAftr]
    target_data = {'service': service, 'binaryData84':binaryData84}

create/import model

In [6]:
input = keras.Input(720, None, name = 'input')
x = layers.Dense(128, activation='sigmoid', use_bias=True, bias_initializer='zeros', name = 'layer_1')(input)
#x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation='sigmoid', use_bias=True, bias_initializer='zeros', name = 'layer_2')(x)

service = layers.Dense(1, activation='sigmoid', name='service')(x)
binaryData = layers.Dense(84, activation='sigmoid', name='binaryData84')(x)

#blockL0 = layers.Dense(1, activation='linear', name='blockL0')(blockL0)
#blockR0 = layers.Dense(1, activation='linear', name='blockR0')(blockR0)
#blockL1 = layers.Dense(1, activation='linear', name='blockL1')(blockL1)
#blockR1 = layers.Dense(1, activation='linear', name='blockR1')(blockR1)
#blockL2 = layers.Dense(1, activation='linear', name='blockL2')(blockL2)
#blockR2 = layers.Dense(1, activation='linear', name='blockR2')(blockR2)

#model = keras.Model(input, [service, blockL0, blockR0, blockL1, blockR1, blockL2, blockR2])
model = keras.Model(input, [service, binaryData])

model.summary()

tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 720)]        0           []                               
                                                                                                  
 layer_1 (Dense)                (None, 128)          92288       ['input[0][0]']                  
                                                                                                  
 layer_2 (Dense)                (None, 128)          16512       ['layer_1[0][0]']                
                                                                                                  
 service (Dense)                (None, 1)            129         ['layer_2[0][0]']                
                                                                                              

compile and learn

In [7]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

In [17]:
loss = 1
while loss >= 0.01:
    fitHistory = model.fit(inputLines, target_data, epochs=1, batch_size=64, validation_split=0.1)
    loss = fitHistory.history['loss'][0]

1407/1407 [==============================] - 38s 27ms/step - loss: 0.0043 - service_loss: 8.8564e-06 - binaryData84_loss: 0.0043 - service_binary_accuracy: 1.0000 - binaryData84_binary_accuracy: 0.9999 - val_loss: 0.0031 - val_service_loss: 2.9020e-06 - val_binaryData84_loss: 0.0031 - val_service_binary_accuracy: 1.0000 - val_binaryData84_binary_accuracy: 1.0000


add boolean convert to model

In [24]:
#complete the model
binaryData2 = layers.Dense(6, activation='linear', trainable = False, name='binaryData')(tf.round(binaryData))
modelP2 = keras.Model(input, [service, binaryData2])

modelP2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 720)]        0           []                               
                                                                                                  
 layer_1 (Dense)                (None, 128)          92288       ['input[0][0]']                  
                                                                                                  
 layer_2 (Dense)                (None, 128)          16512       ['layer_1[0][0]']                
                                                                                                  
 binaryData84 (Dense)           (None, 84)           10836       ['layer_2[0][0]']                
                                                                                            

In [25]:
for layer in model.layers:
    for layerP2 in modelP2.layers:
        if layer.name == layerP2.name:
            layerP2.set_weights(layer.get_weights())
            break

config bits array to int layer

In [26]:
w = np.zeros(binaryData2.node.layer.get_weights()[0].shape[0]*6).reshape(binaryData2.node.layer.get_weights()[0].shape)
bits = 14
for j in range(0, 6):
    for i in range(0, bits):
        w[i+(j*14)][j] = 1 << (bits - i) - 1
        
binaryData2.node.layer.set_weights([w, binaryData2.node.layer.get_weights()[1]])

In [27]:
modelP2.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

In [28]:
tf.round(model.predict(np.array([inputLines[50]]))[1])

<tf.Tensor: shape=(1, 84), dtype=float32, numpy=
array([[1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
        0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)>

In [29]:
modelP2.predict(np.array([inputLines[50]]))

[array([[0.9908153]], dtype=float32),
 array([[10580.,  9865.,  6407.,     0.,     0.,     0.]], dtype=float32)]

save

In [30]:
modelP2.save("model")

INFO:tensorflow:Assets written to: model\assets
